# Map Visualizations


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import geopandas as gpd
import plotly.express as px
from shapely.geometry import Polygon

In [3]:
entrance_df = pd.read_json('../accessibility_score/entrance_score.json', orient='index')
centroid_df = pd.read_json('../accessibility_score/centroid_score.json', orient='index')
census_bbox = gpd.read_file('../get_population_data/census_final_data.json',driver= 'GeoJSON')
park_catchment_areas = gpd.read_file('../get_isodistances/isodistances_entrance_park_coordinates.json',driver= 'GeoJSON')
park_centroid_catchment_areas = gpd.read_file('../get_isodistances/isodistances_entrance_park_coordinates.json',driver= 'GeoJSON')
park_areas = gpd.read_file('../get_park_data/data.json',driver= 'GeoJSON')


In [4]:
def map_one_park_polygons(park_df,park_name,census):
    park = park_df[park_df.name == park_name]
    map = folium.Map(location=[53.41535, 9.90225], tiles="CartoDB Positron", zoom_start=11, control_scale = True)
    for _, r in census.iterrows():
        sim_geo_ = gpd.GeoSeries(r["geometry"])
        geo_j_ = sim_geo_.to_json()
        geo_j_ = folium.GeoJson(data=geo_j_, style_function=lambda x: {"fillColor": "orange"})
        folium.Popup(r["row_number"]).add_to(geo_j_)
        geo_j_.add_to(map)
    for _, r in park.iterrows():
        sim_geo = gpd.GeoSeries(r["geometry"])
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "green"})
        if 'name' in park.columns:
            folium.Popup(r["name"]).add_to(geo_j)
        geo_j.add_to(map)
    
    return map

In [5]:
def map_multiple_polygons(park,census):
    map = folium.Map(location=[53.41535, 9.90225], tiles="CartoDB Positron", zoom_start=11, control_scale = True)
    for _, r in census.iterrows():
        sim_geo_ = gpd.GeoSeries(r["geometry"])
        geo_j_ = sim_geo_.to_json()
        geo_j_ = folium.GeoJson(data=geo_j_, style_function=lambda x: {"fillColor": "orange"})
        if 'row_number' in census.columns:
            folium.Popup(r["row_number"]).add_to(geo_j_)
        geo_j_.add_to(map)
    for _, r in park.iterrows():
        sim_geo = gpd.GeoSeries(r["geometry"])
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "green"})
        if 'name' in park.columns:
            folium.Popup(r["name"]).add_to(geo_j)
        geo_j.add_to(map)
    
    return map

In [6]:
def map_catchment_heatmap(park, catchment_area, park_name, census, sociodemographics):
    map = folium.Map(location=[53.554831, 10.000305], tiles="CartoDB Positron", zoom_start=11, control_scale = False)
    
    # buffering arround park catchment area to get nearest census blocks
    catchment_park_df = catchment_area[catchment_area["name"] == park_name].reset_index()
    park_3395 = catchment_park_df.to_crs(epsg=3395)
    bbox_park = park_3395.buffer(1720,cap_style = 1)

    # convert buffered polygon back to 4326 crs
    envgdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(bbox_park))
    bbox_4326 = gpd.GeoDataFrame(envgdf, crs='EPSG:3395')
    bbox_4326 = bbox_4326.to_crs(epsg=4326)
    bbox_4326["name"] = catchment_park_df["name"]
    bbox_4326

    # find subset of census blocks intersecting with buffered catchment area
    intersecting_blocks = gpd.sjoin(census,bbox_4326,how = "inner", predicate="intersects")

    # convert census df to GeoJSON
    census_gjson = intersecting_blocks.to_json()

    # create choropleth and add to map
    folium.Choropleth(
        geo_data = census_gjson,
        name="choropleth",
        data=intersecting_blocks,
        columns =["row_number", sociodemographics],
        key_on = "feature.properties.row_number",
        fill_color="YlOrBr",
        fill_opacity=0.8,
        line_opacity=0.6,
        legend_name="Average Age"
    ).add_to(map)
    
    sozio = folium.features.GeoJson(
    intersecting_blocks, 
    control=False,
    tooltip=folium.features.GeoJsonTooltip(
        fields=[sociodemographics],
        aliases=[sociodemographics]
        ))
    
    # adding catchment area,reduced by park area to map
    park_area = park[park["name"] == park_name].reset_index()
    difference = catchment_park_df["geometry"].difference(park_area["geometry"])
    difference = folium.GeoJson(data=difference, style_function=lambda x: {"fillColor": "green","fillOpacity": 0.9, "color": "green", "stroke":True,"weight":0.5})
    
    difference.add_to(map)
    
    #map.add_child(sozio)
    map.keep_in_front(difference)
    return map

In [7]:
map_catchment_heatmap(park_areas, park_catchment_areas, "Berner Gutspark", census_bbox, "Alter_D")